<a href="https://colab.research.google.com/github/SHodapp117/Applied-Machine-Learning/blob/main/BigQuery_bquxjob_1abc5b1_18b907cef1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'dapperlabs-data' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=dapperlabs-data:US:bquxjob_1abc5b1_18b907cef1d)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_1abc5b1_18b907cef1d') # Job ID inserted based on the query results selected to explore
print(job.query)

WITH listing_history as (
  select *
  from `dapperlabs-data.berkeley_ds_sandbox.berkeley_ds_source_nfl_historical_listings_time_series`
),
-- get unique rows for every week in last 6m per moment
weekly_series as (
    select distinct a.flow_moment_id, a.moment_flow_edition_id, date_trunc(date, week) as week
    from listing_history as a,
        unnest(generate_date_array(date_trunc(date_sub(date_trunc(current_date, month), interval 6 month), week), date_sub(date_trunc(current_date(), week), interval 3 week), interval 1 day)) as date
),
--- weekly avg of sold listings
sold_avg AS (
  SELECT w.week, w.flow_moment_id, w.moment_flow_edition_id, AVG(l.listing_price_usd) AS avg_sold
  FROM weekly_series as w
  LEFT JOIN (select * from listing_history where listing_status = 'SOLD')  as l
    on w.flow_moment_id = l.flow_moment_id
      and date_trunc(l.event_timestamp, week) = w.week
  GROUP BY w.week, w.flow_moment_id, w.moment_flow_edition_id
),
-- weekly avg of non sold listings
listed_a

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1abc5b1_18b907cef1d') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,week,flow_moment_id,moment_flow_edition_id,avg_sold,avg_listed,edition_floor_listed,moment_value_current_week,target_moment_value_next_week,moment_play_player_position,position_QB,position_RB,position_WR,position_TE,position_LB,position_DL,position_DB,rarity,final_player_score,serial_to_mint_ratio,listed_supply
0,2023-04-30,1000002,547,NaN,10000.0,109.0,109.0,122.0,TE,0,0,0,1,0,0,0,0.009174,0.058625,0.347052,284
1,2023-04-30,100001,362,NaN,NaN,1.0,1.0,1.0,LB,0,0,0,0,1,0,0,1.000000,NaN,0.964900,<NA>
2,2023-04-30,1000014,547,NaN,NaN,109.0,109.0,122.0,TE,0,0,0,1,0,0,0,0.009174,0.058625,0.360400,<NA>
3,2023-04-30,1000017,547,NaN,NaN,109.0,109.0,122.0,TE,0,0,0,1,0,0,0,0.009174,0.058625,0.363737,<NA>
4,2023-04-30,1000018,547,NaN,130.0,109.0,130.0,130.0,TE,0,0,0,1,0,0,0,0.009174,0.058625,0.364850,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16592995,2023-10-08,999964,547,NaN,NaN,46.0,46.0,NaN,TE,0,0,0,1,0,0,0,0.021739,0.058625,0.304783,<NA>
16592996,2023-10-08,999967,547,NaN,NaN,46.0,46.0,NaN,TE,0,0,0,1,0,0,0,0.021739,0.058625,0.308120,<NA>
16592997,2023-10-08,999973,547,NaN,NaN,46.0,46.0,NaN,TE,0,0,0,1,0,0,0,0.021739,0.058625,0.314794,<NA>
16592998,2023-10-08,999979,547,NaN,95.0,46.0,95.0,NaN,TE,0,0,0,1,0,0,0,0.021739,0.058625,0.321468,30


In [4]:
df = results[['position_QB','position_RB',	'position_WR',	'position_TE',	'position_LB',	'position_DL',	'position_DB',	'rarity',	'final_player_score',	'serial_to_mint_ratio',	'listed_supply','flow_moment_id' ,'avg_sold',	'avg_listed',	'edition_floor_listed',	'moment_value_current_week',	'target_moment_value_next_week']]


In [5]:
numerical_features = ['final_player_score', 'serial_to_mint_ratio', 'listed_supply', 'avg_sold', 'avg_listed', 'moment_value_current_week','rarity']
df.describe()

,position_QB,position_RB,position_WR,position_TE,position_LB,position_DL,position_DB,rarity,final_player_score,serial_to_mint_ratio,listed_supply,avg_sold,avg_listed,edition_floor_listed,moment_value_current_week,target_moment_value_next_week
count,3000000.0,3000000.0,3000000.0,3000000.0,3000000.0,3000000.0,3000000.0,2.947603e+06,1.743559e+06,3.000000e+06,854768.0,133185.000000,735444.000000,2.947603e+06,2.947603e+06,2.949742e+06
mean,0.141126,0.163627,0.286874,0.070292,0.073004,0.074915,0.139387,6.221609e-01,1.668771e-01,4.903003e-01,208.807584,14.287783,1972.241418,1.885955e+01,2.666735e+01,2.687868e+01
std,0.348152,0.369937,0.452302,0.255639,0.260143,0.263254,0.34635,3.758400e-01,1.969453e-01,2.949105e-01,107.562818,79.446876,32830.194050,1.685044e+03,1.708085e+03,1.748163e+03
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000e-06,4.260646e-03,1.000000e-04,1.0,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.333333e-01,4.168059e-02,2.320000e-01,93.0,2.000000,8.000000,1.000000e+00,1.000000e+00,1.000000e+00
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000e-01,7.502917e-02,4.903000e-01,284.0,3.000000,20.000000,2.000000e+00,2.000000e+00,2.000000e+00
75%,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000e+00,1.620124e-01,7.468571e-01,284.0,5.000000,69.000000,3.000000e+00,5.000000e+00,5.000000e+00
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000e+00,7.036322e-01,1.000000e+00,284.0,7500.000000,1000000.000000,1.000000e+06,1.000000e+06,1.000000e+06


In [6]:
# Count the number of missing values in each column of the DataFrame
missing_values_count = df.isna().sum()


# Print the counts
print("Missing values in each column:")
print(missing_values_count)

Missing values in each column:
position_QB                            0
position_RB                            0
position_WR                            0
position_TE                            0
position_LB                            0
position_DL                            0
position_DB                            0
rarity                             52397
final_player_score               1256441
serial_to_mint_ratio                   0
listed_supply                    2145232
flow_moment_id                         0
avg_sold                         2866815
avg_listed                       2264556
edition_floor_listed               52397
moment_value_current_week          52397
target_moment_value_next_week      50258
dtype: int64


In [7]:
df = df.astype(float)
column_dtypes = df.dtypes
print(column_dtypes)
df = df.dropna()

position_QB                      float64
position_RB                      float64
position_WR                      float64
position_TE                      float64
position_LB                      float64
position_DL                      float64
position_DB                      float64
rarity                           float64
final_player_score               float64
serial_to_mint_ratio             float64
listed_supply                    float64
flow_moment_id                   float64
avg_sold                         float64
avg_listed                       float64
edition_floor_listed             float64
moment_value_current_week        float64
target_moment_value_next_week    float64
dtype: object


In [8]:
df.describe()

,position_QB,position_RB,position_WR,position_TE,position_LB,position_DL,position_DB,rarity,final_player_score,serial_to_mint_ratio,listed_supply,flow_moment_id,avg_sold,avg_listed,edition_floor_listed,moment_value_current_week,target_moment_value_next_week
count,281.000000,281.000000,281.000000,281.000000,281.0,281.0,281.0,281.000000,281.000000,281.000000,281.000000,2.810000e+02,281.000000,281.000000,281.000000,281.000000,281.000000
mean,0.195730,0.202847,0.501779,0.099644,0.0,0.0,0.0,0.564842,0.165520,0.424091,87.451957,2.987423e+06,27.010439,9088.679092,20.035587,27.010439,34.155397
std,0.397469,0.402837,0.500889,0.300059,0.0,0.0,0.0,0.401931,0.189971,0.299847,54.693092,1.393645e+06,79.261394,86310.528828,65.771185,79.261394,113.378900
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.001565,0.004261,0.000286,1.000000,5.051000e+03,1.000000,2.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.052581,0.156506,30.000000,1.890911e+06,2.000000,3.000000,1.000000,2.000000,2.000000
50%,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.500000,0.081239,0.390800,98.000000,3.120238e+06,3.000000,6.000000,2.000000,3.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1.000000,0.159247,0.662971,140.000000,4.145431e+06,16.000000,36.200000,6.000000,16.000000,17.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000,0.703632,0.995400,165.000000,4.920077e+06,675.000000,1000000.000000,639.000000,675.000000,1000.000000


In [3]:
import pandas as pd
import numpy as np

# Define the threshold percentile (e.g., 75th percentile)
percentile_threshold = 75

# Create a list to store the filtered dataframes for each numerical column
filtered_dataframes = []

# Iterate over your numerical features
for numerical_feature in numerical_features:
    # Calculate the percentile for the current numerical column
    percentile_value = np.percentile(df[numerical_feature], percentile_threshold)

    # Filter the data to keep only data points above the percentile value for the current column
    filtered_data = df[df[numerical_feature] >= percentile_value]

    # Append the filtered dataframe to the list
    filtered_dataframes.append(filtered_data)

# Now, you have a list of dataframes, each containing the data points above the 75th percentile for a specific numerical feature.

# If you want to combine these dataframes, you can do so using pandas.concat, for example:
filtered_data_combined = pd.concat(filtered_dataframes, axis=1)

# You can then use filtered_data_combined to create your training and testing datasets.


NameError: ignored

## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [8]:
# Import the libraries we'll use below.
from matplotlib import pyplot as plt
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import tensorflow as tf

In [2]:
#I want to use SGD for the optimizer here, so I am willing to create a Core Assumption for a V1 model CA1{The ground truth value does not appricable change over time, so we can mix the indicies and ignore the time dimention}
moment_data = df.reindex(np.random.permutation(df.shape[0]))
moment_data = moment_data.dropna(0)


NameError: ignored

In [ ]:
# Training test split

# We'll use these input features.
features = ['position_QB', 'position_RB', 'position_WR', 'position_TE', 'position_LB', 'position_DL', 'position_DB', 'rarity', 'final_player_score',
            'serial_to_mint_ratio', 'listed_supply', 'avg_sold', 'avg_listed', 'moment_value_current_week']

# Use a ~80/20 train/test split.
moment_train = moment_data[:1000000]
moment_test = moment_data[1000000:12000000]

# Create separate variables for features (inputs) and labels (outputs).
# We will be using these in the cells below.
moment_train_features = moment_train[features]
moment_test_features = moment_test[features]
moment_train_labels = moment_train['target_moment_value_next_week']
moment_test_labels = moment_test['target_moment_value_next_week']

# Confirm the data shapes are as expected.
print('train data shape:', moment_train_features.shape)
print('train labels shape:', moment_train_labels.shape)
print('test data shape:', moment_test_features.shape)
print('test labels shape:', moment_test_labels.shape)


In [ ]:
numerical_features = ['final_player_score', 'serial_to_mint_ratio', 'listed_supply', 'avg_sold', 'avg_listed', 'moment_value_current_week','rarity']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# List of numerical features to be scaled
numerical_features = ['final_player_score', 'serial_to_mint_ratio', 'listed_supply', 'avg_sold', 'avg_listed', 'moment_value_current_week', 'rarity']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the scaler on the training data
moment_train_features[numerical_features] = scaler.fit_transform(moment_train_features[numerical_features])

# Transform the test data using the same scaler
moment_test_features[numerical_features] = scaler.transform(moment_test_features[numerical_features])

# Check the scaled data
print('Min-Max scaled train data:')
print(moment_train_features.head())

print('Min-Max scaled test data:')
print(moment_test_features.head())


In [ ]:
#making pairwise corr coefficents
# Calculate pairwise correlation coefficients for the training data
correlation_matrix = moment_train.corr()

# Print the correlation matrix
print(correlation_matrix)



In [ ]:
#numerical_features = ['final_player_score', 'serial_to_mint_ratio', 'listed_supply', 'avg_sold', 'avg_listed', 'moment_value_current_week','rarity']
#categorical_features = ['position_QB', 'position_RB', 'position_WR', 'position_TE', 'position_LB', 'position_DL', 'position_DB', ]

In [ ]:
def build_model(num_features, learning_rate):
  """Build a TF linear regression model using Keras.
  Args:
    num_features: The number of input features.
    learning_rate: The desired learning rate for SGD.
  Returns:
    model: A tf.keras model (graph).
  """
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      units=1,                     # output dim
      input_shape=[num_features],  # input dim
      use_bias=True,               # use a bias (intercept) param
      kernel_initializer=tf.ones_initializer,  # initialize params to 1
      bias_initializer=tf.ones_initializer,    # initialize bias to 1
  ))


  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)


  model.compile(loss='mse', optimizer=optimizer)
  return model

'position_QB', 'position_RB', 'position_WR', 'position_TE', 'position_LB', 'position_DL', 'position_DB',

In [ ]:
model = build_model(num_features=6, learning_rate=0.00000001)

history = model.fit(
  x = moment_train_features[[ 'rarity', 'final_player_score',
            'serial_to_mint_ratio', 'listed_supply', 'avg_listed', 'moment_value_current_week']],
  y = moment_train_labels,
  validation_split=0.1,  # use 10% of the examples as a validation set
  epochs=1500,
  batch_size=10000,
  verbose=0)

history = pd.DataFrame(history.history)
display(history)